In [1]:
%cd ../
%matplotlib inline

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

import csv
import glob
import logging
import os
import re

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

db_file = 'data/sherlock_merlin.db'
mask_file = '/home/eli/Documents/Sherlock_Merlin/wholebrain.nii.gz'

OVERWRITE = True

In [4]:
existed = os.path.isfile(db_file)
db = niidb.FMriActivationsDb(db_file, mask=mask_file)

if not existed or OVERWRITE:
    data_files = glob.glob('/home/eli/Documents/Sherlock_Merlin/sub-*/func/*.nii.gz')
    regex = re.compile("sub-([0-9]{1,2})_task-([A-Za-z]+)(_run-[0-9]{1,2})?_bold.nii.gz")
    for data_file in data_files:
        m = regex.search(data_file)
        subject, task, run = m.groups()
        if run is None:
            run = ''
        tsv_file = '/sub-%s_task-%s%s_events.tsv' % (subject, task, run)
        tsv_file = os.path.dirname(data_file) + tsv_file
        with open(tsv_file, 'r') as tsv:
            tsv_reader = csv.DictReader(tsv, dialect='excel-tab')
            onset, duration, stimfile = [row for row in tsv_reader][0].values()
            onset = round(float(onset))
            duration = round(float(duration))
        
        logging.info("Recording blocks in %s", data_file)
        
        start_block = niidb.FMriActivationBlock()
        start_block.task = 'rest'
        start_block.filename = data_file
        start_block.subject = subject
        start_block.start_time = 0
        start_block.end_time = onset - 1
        db.upsert(start_block)
        logging.info('Initial rest from until stimulus onset at %d', onset)
        
        task_block = niidb.FMriActivationBlock()
        task_block.task = stimfile
        task_block.filename = data_file
        task_block.subject = subject
        task_block.start_time = onset
        task_block.end_time = onset + duration
        db.upsert(task_block)
        logging.info('Stimulus from %d to %d', onset, onset + duration)
        
        end_block = niidb.FMriActivationBlock()
        end_block.task = stimfile
        end_block.filename = data_file
        end_block.subject = subject
        end_block.start_time = onset + duration + 1
        end_block.end_time = None
        db.upsert(end_block)
        logging.info('Final rest from %d until end of run', onset + duration)

05/12/2018 23:05:59 Recording blocks in /home/eli/Documents/Sherlock_Merlin/sub-27/func/sub-27_task-MerlinMovie_bold.nii.gz
05/12/2018 23:05:59 Initial rest from until stimulus onset at 26
05/12/2018 23:05:59 Stimulus from 26 to 1528
05/12/2018 23:05:59 Final rest from 1528 until end of run
05/12/2018 23:05:59 Recording blocks in /home/eli/Documents/Sherlock_Merlin/sub-27/func/sub-27_task-SherlockAudio_bold.nii.gz
05/12/2018 23:05:59 Initial rest from until stimulus onset at 3
05/12/2018 23:05:59 Stimulus from 3 to 1080
05/12/2018 23:05:59 Final rest from 1080 until end of run
05/12/2018 23:05:59 Recording blocks in /home/eli/Documents/Sherlock_Merlin/sub-04/func/sub-04_task-SherlockMovie_bold.nii.gz
05/12/2018 23:05:59 Initial rest from until stimulus onset at 26
05/12/2018 23:05:59 Stimulus from 26 to 1442
05/12/2018 23:05:59 Final rest from 1442 until end of run
05/12/2018 23:05:59 Recording blocks in /home/eli/Documents/Sherlock_Merlin/sub-04/func/sub-04_task-MerlinAudio_bold.nii.g

In [5]:
logging.info('Finished building NiiDb out of Sherlock_Merlin dataset')

05/12/2018 23:06:11 Finished building NiiDb out of Sherlock_Merlin dataset
